# HW2 - Doing Excel work with Python - Analysis 2 - PDalvi

We have created the project structure using cookiecutter-datascience-simple template, copied the logs folder (i.e input files) to the data folder and put the required files and project folder under version control.

We start by importing the required python libraires for our analysis.

In [2]:
import csv
import openpyxl
import os
import glob   
import pandas as pd

We are hardcoding the output file in the first part of the analysis, and creating a blank BCM2.xlsx file in the output folder, and then we are accessing the logs folder to get the names of the files to be merged in single xlsx file.

In [3]:
wb = openpyxl.Workbook()
wb.save('./output/BCM2.xlsx')

path = "./data/logs"
all_files = glob.glob(os.path.join(path, "*.csv"))
all_files

['./data/logs\\BCM-E-tCenter-Deep.csv',
 './data/logs\\BCM-E-tCenter-Medium.csv',
 './data/logs\\BCM-E-tCenter-Shallow.csv',
 './data/logs\\BCM-E-tLeft-Deep.csv',
 './data/logs\\BCM-E-tLeft-Medium.csv',
 './data/logs\\BCM-E-tRight-Deep.csv',
 './data/logs\\BCM-E-tRight-Medium.csv',
 './data/logs\\BCM-N-tCenter-Deep.csv',
 './data/logs\\BCM-N-tCenter-Medium.csv',
 './data/logs\\BCM-N-tLeft-Deep.csv',
 './data/logs\\BCM-N-tLeft-Medium.csv',
 './data/logs\\BCM-N-tRight-Deep.csv',
 './data/logs\\BCM-N-tRight-Medium.csv',
 './data/logs\\BCM-N-tRight-Shallow.csv']

#### Consolidation

Next we loop through the input files, create a sheet in the output file with the sheetname same as the input file name and past the contents in the excel worksheet. In this process we all do some formatting in the worksheet like adding the column headers and the datetime column specifically was getting copied as string so we formatted it to datetime in the dataframe itself before copying it to the output excel worksheet. Lastly, the important step was to save the excel once all the worksheets were copied and formatted.

In [4]:
writer = pd.ExcelWriter('./output/BCM.xlsx', engine='xlsxwriter')

for f in all_files:
    #print(f)
    column_names=["datetime", "scale", "temperature"]
    df = pd.read_csv(f)
    #df = pd.read_csv(f, parse_dates=[0])
    df.columns = column_names

    df['datetime'] = df['datetime'].astype('datetime64[ns]')
    #print(df.dtypes)
    #print(df["datetime"])

    #print(os.path.basename(f)[:-4])
    
    df.to_excel(writer, sheet_name=os.path.basename(f)[:-4], index=False)
    
writer.save()

#### Summarization

For adding the summary of the temperature and the datetime, we open and load our output workbook, loop through each of our worksheets and add text and formulas respectively in each sheet. Here i just gave manual cell address for G2 accessed by row = 2, column = 7 for the min_temp and the next values are added respectively. For accessing the end cell address for our formulas, I used max_row which gives the max of the row numbers in the active sheet.

In [5]:
#from openpyxl.formula.translate import

wb = openpyxl.Workbook()
wb = openpyxl.load_workbook(filename = './output/BCM.xlsx')

for sheet in wb.worksheets:
    #print(sheet)
    rowcnt = sheet.max_row
    
    mintemp = "= min(C2:C{})".format(rowcnt)
    maxtemp = "= max(C2:C{})".format(rowcnt)
    meantemp = "= average(C2:C{})".format(rowcnt)
    
    mindate = "= TEXT(min(A2:A{}), \"yyyy-mm-dd hh:mm:ss\")".format(rowcnt)
    #print(mindate)
    maxdate = "= TEXT(max(A2:A{}), \"yyyy-mm-dd hh:mm:ss\")".format(rowcnt)
    
    #print(mintemp)
    #print(rowcnt)
    sheet.cell(row = 2, column = 7, value = 'min_temp')
    sheet.cell(row = 3, column = 7, value = 'max_temp')
    sheet.cell(row = 4, column = 7, value = 'mean_temp')
    
    sheet.cell(row = 6, column = 7, value = 'min_date')
    sheet.cell(row = 7, column = 7, value = 'max_date')
   
    sheet.cell(row = 2, column = 8, value = mintemp)
    sheet.cell(row = 3, column = 8, value = maxtemp)
    sheet.cell(row = 4, column = 8, value = meantemp)
    
    sheet.cell(row = 6, column = 8, value = mindate)
    sheet.cell(row = 7, column = 8, value = maxdate)
    
    
wb.save('./output/BCM.xlsx')

For the Extra credit problem part, I first created a new hacker_logs folder that holds all the input files and renamed few of the BCM files to JEF and MAE. Next I found all the filesnames in the hacker_logs path and created a list of elements with the 19-22 character from the file name which happen to be stream name, then by set function of list got distinct elements of the stream names. Once we have the distinct stream names, I created the xlsx files for all the streams in output/hacker_output folder. Next, we run a for loop of the stream files and find all the csv files with the particular stream and copy them into the output xlxs file in respective sheets. After this, the processing and formatting of the dataframe and xlsx file is the same as we did in the summarization part above.

In [8]:
path = "./data/hacker_logs"
all_files = glob.glob(os.path.join(path, "*.csv"))
all_files

y = []
for f in all_files:
    y.append(f[19:22])
    
y = list(set(y))

op_path = "./output/hacker_output/"
for i in y:
    #print(i)
    opfname = op_path+i+".xlsx"
    #print(opfname)
    wb = openpyxl.Workbook()
    wb.save(opfname)
    
op_all_files = glob.glob(os.path.join(op_path, "*.xlsx"))
#print(op_all_files)


for op in op_all_files:
    #print(op)
    #print(os.path.basename(op)[:3])
    ipfiles = os.path.basename(op)[:3]+'*.csv'
    all_files = glob.glob(os.path.join(path, ipfiles))
    writer = pd.ExcelWriter(op, engine='xlsxwriter')
    for f in all_files:
        #print(f)
        column_names=["datetime", "scale", "temperature"]
        df = pd.read_csv(f)
    
        df.columns = column_names
        df['datetime'] = df['datetime'].astype('datetime64[ns]')
    
        #print(os.path.basename(f)[:3])
        df.to_excel(writer, sheet_name=os.path.basename(f)[:-4], index=False)
    writer.save()

In [10]:
#from openpyxl.formula.translate import

for op in op_all_files:
    #print(op)
    
    wb = openpyxl.Workbook()
    wb = openpyxl.load_workbook(filename = op)

    for sheet in wb.worksheets:
        #print(sheet)
        rowcnt = sheet.max_row
    
        mintemp = "= min(C2:C{})".format(rowcnt)
        maxtemp = "= max(C2:C{})".format(rowcnt)
        meantemp = "= average(C2:C{})".format(rowcnt)
    
        mindate = "= TEXT(min(A2:A{}), \"yyyy-mm-dd hh:mm:ss\")".format(rowcnt)
        #print(mindate)
        maxdate = "= TEXT(max(A2:A{}), \"yyyy-mm-dd hh:mm:ss\")".format(rowcnt)
    
        #print(mintemp)
        #print(rowcnt)
        sheet.cell(row = 2, column = 7, value = 'min_temp')
        sheet.cell(row = 3, column = 7, value = 'max_temp')
        sheet.cell(row = 4, column = 7, value = 'mean_temp')
    
        sheet.cell(row = 6, column = 7, value = 'min_date')
        sheet.cell(row = 7, column = 7, value = 'max_date')
   
        sheet.cell(row = 2, column = 8, value = mintemp)
        sheet.cell(row = 3, column = 8, value = maxtemp)
        sheet.cell(row = 4, column = 8, value = meantemp)
    
        sheet.cell(row = 6, column = 8, value = mindate)
        sheet.cell(row = 7, column = 8, value = maxdate)
    wb.save(op)